# Impact Quantization

In [19]:
# imports
import os
import numpy as np
import pandas as pd
import pickle

## Functions

In [20]:
# function to quantize impact points
def quantize_impact_points(impact_points, time_stamps, quant_params):
    assert len(impact_points) == len(time_stamps)
    n_frames, n_limbs = impact_points.shape
    
    # quantize time stamps
    dt = 60 / (quant_params['bpm'] * quant_params['subdiv'])
    quantized_time_stamps = np.round((time_stamps - quant_params['beat_ref']) / dt) * dt + quant_params['beat_ref']
    impact_points[quantized_time_stamps < 0] = np.zeros((n_limbs), dtype=bool)
    quantized_time_stamps[quantized_time_stamps < 0] = 0
    
    # format quantized impacts
    quantized_impacts = []
    for l in range(n_limbs):
        quantized_impacts.append(np.unique(quantized_time_stamps[impact_points[:,l]]))
    
    return quantized_impacts

## Quantize Impact Points

In [25]:
# load impact points
load_dir = os.path.join('data', 'impact_points')
vid_name = 'j1'
impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))
impact_points.shape, time_stamps.shape

((4606, 4), (4606,))

In [26]:
# quant params for j1
quant_params = {
    'beat_ref': 16.25,
    'bpm': 136,
    'subdiv': 2
}
# run
quantized_impacts = quantize_impact_points(impact_points, time_stamps, quant_params)
quantized_impacts

[array([ 18.67647059,  32.35294118,  32.57352941,  38.52941176,
         38.75      ,  39.19117647,  39.41176471,  39.63235294,
         39.85294118,  48.01470588,  49.11764706,  51.10294118,
         51.32352941,  51.98529412,  53.08823529,  54.19117647,
         54.41176471,  55.73529412,  58.38235294,  59.26470588,
         59.48529412,  60.14705882,  60.36764706,  60.58823529,
         60.80882353,  61.02941176,  61.25      ,  61.47058824,
         62.57352941,  62.79411765,  63.01470588,  63.23529412,
         63.45588235,  63.67647059,  63.89705882,  64.33823529,
         64.55882353,  64.77941176,  65.        ,  65.44117647,
         65.66176471,  65.88235294,  66.10294118,  66.32352941,
         66.54411765,  66.76470588,  67.20588235,  67.42647059,
         67.64705882,  67.86764706,  68.08823529,  68.30882353,
         68.52941176,  68.97058824,  69.19117647,  69.41176471,
         69.63235294,  69.85294118,  70.29411765,  70.51470588,
         70.73529412,  70.95588235,  71.

In [27]:
# save results to file
save_dir = os.path.join('data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
    pickle.dump(quantized_impacts, f)

## Run on All Videos

In [24]:
# run impact detection for all videos
load_dir = os.path.join('data', 'impact_points')
vid_info_df = pd.read_csv(os.path.join('video', 'video_info.csv'), index_col='vid_name')

save_dir = os.path.join('data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# iterate over all videos
for vid_name in vid_info_df.index:
    if vid_name != 'group':
        print(10 * '-')
        print(vid_name)
        
        # load impact points and time stamps
        impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
        time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))
         
        # quantize impact points and save
        quant_params = {
            'beat_ref': vid_info_df.loc[vid_name]['beat_ref'],
            'bpm': vid_info_df.loc[vid_name]['bpm'],
            'subdiv': vid_info_df.loc[vid_name]['subdiv']
        }
        quantized_impacts = quantize_impact_points(impact_points, time_stamps, quant_params)
        with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
            pickle.dump(quantized_impacts, f)

----------
a1
----------
a2
----------
e1
----------
e2
----------
e3
----------
j1
----------
j2
----------
u1
----------
u2
